In [ ]:
import urllib.request as ur
import json
import math
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ete3
import upsetplot

In [ ]:
ncbi = ete3.NCBITaxa()
#ncbi.update_taxonomy_database()


# Functions 

### Score parameters 

In [ ]:
def Parameters (name, default) :
    while True :   
        try:
            y = input(f"What coefficient for {name}? (by default: {default}) ")
            if y ==  "" or y ==  " " :
                y  =  default 
                print(f"Valid value: {default}")
                break
            if  0 <= float(y) <= 1 : 
                print(f"Valid value: {y}")
                break
            else :
                print("Invalid value") 
            continue
        except :
            print("Invalid value") 
            continue
    return(y)


### Score option

In [ ]:
def Score_type () :
    while True :   
        try:
            y = input(f"What type of score ? addition = completeness + consistency or multiplication = completeness + consistency (by default: addition) ")
            if y ==  "addition" or y ==  "addition " or y ==  "Addition" or y ==  "Addition " or y ==  "ADDITION" or y ==  "ADDITION " or y ==  "" or y == " ":
                y  =  "addition" 
                print(f"Valid option: {y}")
                break
            if  y ==  "multiplication" or y ==  "multiplication " or y ==  "Multiplication" or y ==  "Multiplication " or y ==  "MULTIPLICATION" or y ==  "MULTIPLICATION ": 
                print(f"Valid option: {y}")
                break
            else :
                print("Invalid value") 
            continue
        except :
            print("Invalid value") 
            continue
    return(y)


### Sigmoïde transformation 

In [ ]:
def Sigmoide(x, a):
    return (1/(1 + np.exp((-x + 0.5)*int(a)))) # f(x)=((1)/(1+ℯ^(10*(-x+0.5))))


### Scoring the species

In [ ]:
def  Scoring (Data,  parameters, score_option):
    
    cons = float(parameters[0])   
    part_cons = parameters[1]
    fra_cons = parameters[2]
    cont = parameters[3]
    fra_inco = parameters[4]
    part_inco = parameters[5]
    
    Data["prot_consistent_fit"] = Data["prot_consistent"] - Data["prot_consistent_partial"] - Data["prot_consistent_fragment"]
    Data["prot_contamination_fit"] = Data["prot_contamination"] - Data["prot_contamination_partial"] - Data["prot_contamination_fragment"]
    Data["prot_inconsistent_fit"] = Data["prot_inconsistent"] - Data["prot_inconsistent_partial"] - Data["prot_inconsistent_fragment"]

    
    score = list()
    
    for index, row in Data.iterrows(): 
        single = row["hog_single"]
        duplicated = row["hog_duplicated"]
        consistent = row["prot_consistent"]
        consistent_fit = row["prot_consistent_fit"]
        partial_cons = row["prot_consistent_partial"]
        fragmented_cons = row["prot_consistent_fragment"]
        contamination = row["prot_contamination"]
        frangmented_incons = row["prot_inconsistent_fragment"]
        partial_incons = row["prot_inconsistent_partial"]
        
        completeness = single + duplicated 
        
        consistency_plus = consistent_fit*cons + partial_cons*part_cons + fragmented_cons*fra_cons # ajouter signle et duplicated et pas besoin de faire - missing puisque deduit 
        consistency_minus = contamination*cont + frangmented_incons*fra_inco + partial_incons*part_inco 
        consistency = consistency_plus - consistency_minus
        
        if score_option == "addition":
            score_tot = (completeness + consistency)/200
        elif score_option == "multiplication":
            score_tot = (completeness * consistency)/10000
        
        score_sigmo = Sigmoide(score_tot, sig_coef) # call  the sigmoïde coefficient given by the user 
        
        score.append(score_sigmo)
    
    Data["Score"] = score
    
    return (Data)


### Transform a list column into a dataframe column

In [ ]:
def DF_colomn (z):
    z.tolist()
    colomn = pd.DataFrame(z)
    colomn["index"] = range(0, len(colomn), 1)
    colomn = colomn.set_index("index")
    return(colomn)

### Transform a dataframe column into a list

In [ ]:
def Col_in_list (colname, dtframe): 
    n_list = np.unique(dtframe[colname])
    n_list = list(n_list)
    return(n_list)

### Add the species names in the column "sc_name"

In [ ]:
def Add_names (Data):
    dic_names = ncbi.get_taxid_translator(Data["taxon_id"])
    names = list()

    for index, row in Data.iterrows(): 
        t_id = row["taxon_id"]
        names.append(dic_names.get(t_id))
        
    Data["sc_name"] = names
    return()

### Find the best score of each species in a database

In [ ]:
def Best_scores_per_dtbase (dtbase, dtframe):
    
    dtframe.sort_values('Score',ascending=False, inplace=True) 
    
    l_dtbase_tax = list()
    l_dtbase_scores = list()
    l_tax_dtbase = list()
    
    keep_data1 = pd.DataFrame(columns = [dtframe.columns])
    
    for index, row in dtframe.iterrows():    # list taxon of the database and keep the score associated
        if row["source_db"] == dtbase:
            l_dtbase_tax.append(row["taxon_id"])
            l_dtbase_scores.append(row["Score"])
            keep_data1.loc[index] = row.tolist()
    
    best_scores = pd.DataFrame(columns = [dtframe.columns])
    
    for index, row in keep_data1.iterrows():     # list only the best 
        if row["taxon_id"] not in l_tax_dtbase:
            best_scores.loc[index] = row.tolist()
            l_tax_dtbase.append(row["taxon_id"])
    
    return(best_scores)


### Find the best database for each commun taxon

In [ ]:
def Best_dtbase_per_taxon (b_s_dtbase_1, b_s_dtbase_2, b_s_dtbase_3, l_dtbase, result):
    
    l_tax_3dtbase = list()
    dtbase_tax = pd.DataFrame(columns = [result.columns])

                        #keep_data1.loc[index] = row.tolist()
    
    for index, row in b_s_dtbase_1.iterrows():     # find the commun taxons
        if int(row["taxon_id"]) in Col_in_list("taxon_id", b_s_dtbase_2) and int(row["taxon_id"]) in Col_in_list("taxon_id", b_s_dtbase_3): 
            l_tax_3dtbase.append(int(row["taxon_id"]))
            dtbase_tax.loc[index] = row.tolist()

    l_tax = [int(item) for item in l_tax_3dtbase]
    l_tax_1 = l_tax.copy()
    l_tax_2 = l_tax.copy()
    l_tax_3 = l_tax.copy()
     
    common_tax_1 = pd.DataFrame(columns = result.columns)
    common_tax_2 = pd.DataFrame(columns = result.columns)
    common_tax_3 = pd.DataFrame(columns = result.columns)
    
    for index, row in dtbase_tax.iterrows(): # select the commun taxon in the first dtbase
        if row["taxon_id"] in l_tax_1:
            common_tax_1.loc[index] = row.tolist()
            l_tax_1.remove(row["taxon_id"])

    common_tax_1.sort_values(by = ["taxon_id"], inplace = True)
    common_tax_1.reset_index(drop = True, inplace = True)
        
    for index, row in b_s_dtbase_2.iterrows(): # select the commun taxon in the second dtbase
        if row["taxon_id"] in l_tax_2:
            common_tax_2.loc[index] = row.tolist()
            l_tax_2.remove(row["taxon_id"])

    common_tax_2.sort_values(by = ["taxon_id"], inplace = True)            
    common_tax_2.reset_index(drop = True, inplace = True)
    
    for index, row in b_s_dtbase_3.iterrows(): # select the commun taxon in the third dtbase
        if row["taxon_id"] in l_tax_3:
            common_tax_3.loc[index] = row.tolist()
            l_tax_3.remove(row["taxon_id"])

    common_tax_3.sort_values(by = ["taxon_id"], inplace = True)
    common_tax_3.reset_index(drop = True, inplace = True)
    
    best_dt_tax = pd.DataFrame(index = l_tax, columns = l_dtbase)
    best_dt_tax = pd.concat([common_tax_1, common_tax_2], ignore_index=True)
    best_dt_tax = pd.concat([best_dt_tax, common_tax_3], ignore_index=True)
    best_dt_tax.sort_values(by = ["taxon_id", "Score"], inplace = True, ascending = [True, False]) 
    
    iter_dt_tax = best_dt_tax.copy()
    l_tax_4 = l_tax.copy()
    
    for index, row in iter_dt_tax.iterrows():
        if row["taxon_id"] in l_tax_4:
            l_tax_4.remove(row["taxon_id"])
        else:
            best_dt_tax.drop([index], axis=0, inplace=True)
    
    best_dt_tax.sort_values(by = ["Score"], inplace = True, ascending = False)
    
    tax_b_s = pd.DataFrame(columns =  [l_dtbase[2], l_dtbase[0], l_dtbase[1]])
    tax_b_s[l_dtbase[2]] = common_tax_1["Score"]
    tax_b_s[l_dtbase[0]] = common_tax_2["Score"]
    tax_b_s[l_dtbase[1]] = common_tax_3["Score"]
    
    max_score = tax_b_s.max(axis = 1)
    max_score = [float(item) for item in max_score]
    
    max_dtbase = list()
    
    for index, row in tax_b_s.iterrows(): 
        if str(max_score[index]) == str(tax_b_s[l_dtbase[2]][index]):
            max_dtbase.append(l_dtbase[2])
        elif max_score[index] == tax_b_s[l_dtbase[0]][index]:
            max_dtbase.append(l_dtbase[0])
        elif max_score[index] == tax_b_s[l_dtbase[1]][index]:
            max_dtbase.append(l_dtbase[1])
    
    tax_b_s["max_score"] = max_score
    tax_b_s["best_dtbase"] = max_dtbase
    tax_b_s["taxon_id"] = common_tax_1["taxon_id"]
    
    tax_b_s['diff_UP_Ens'] = tax_b_s['UniProt Reference Proteomes'] - tax_b_s['Ensembl']
    tax_b_s['diff_UP_RS'] = tax_b_s['UniProt Reference Proteomes'] - tax_b_s['RefSeq']
    tax_b_s['diff_Ens_RS'] = tax_b_s['Ensembl'] - tax_b_s['RefSeq']
    
    return (best_dt_tax, tax_b_s) 

# "User interface"

### Taxon id to study (and verification)

In [ ]:
# Work with 1963758 and 1437183

while True :   
    try:
        tax_id = input("What is the NCBI id of the taxa to analyze? ")
        if len(ncbi.get_taxid_translator([tax_id])) != 0: 
            print("Valid taxon id")
            break
        else :
            print("Invalid taxon id") 
        continue
    except :
        print("Invalid taxon id") 
        continue


### Sigmoïde coefficient to score more or less thigher the species 

In [ ]:
# Either 8 or 10

while True :   
    try:
        sig_coef = input("What coefficient for the sigmoïde transformation of scores? (8 or 10, 10 tighter than 8) ")
        if sig_coef ==  "" or sig_coef ==  " " :
            sig_coef  =  4 
            print(f"By default coeficient: {sig_coef}")
            break
        if  int(sig_coef) == 4 or int(sig_coef) == 8 or int(sig_coef) == 10 : # voir pour adoucir avec 6 ou 4 
            print("Valid value")
            break
        else :
            print("Invalid value") 
        continue
    except :
        print("Invalid value") 
        continue
    

### Parameters of score 

In [ ]:
# By default : (1, 0.5, 0.5, 0.8, 0.5, 0.5)

score_option = Score_type()
cons = Parameters("consistent proteome", 1)
part_cons = Parameters("partially consistent proteome", 0.5)
fra_cons = Parameters("fragmented consistent proteome", 0.5)
cont = Parameters("contamination", 0.8)
fra_inco = Parameters("fragmented and incomplete proteome", 0.5)
part_inco = Parameters("partial and incomplete proteome", 0.5)
parameters = (cons, part_cons, fra_cons, cont, fra_inco, part_inco)
parameters = [float(x) for x in parameters]

# Main - part 1 : scoring the species

In [ ]:
# glires 314147  
# mesagiospermae 1437183

#adress = f"http://localhost:8000/api/results/?taxon_ids={tax_id}"

#
adress = f"https://omark.omabrowser.org/api/results/?taxon_ids={tax_id}"

# Get the data 
data_row = ur.urlopen(adress)
data_json = json.loads(data_row.read())
Data = pd.DataFrame.from_dict(data_json)

#Score the genomes of interest 
result = Scoring(Data, parameters, score_option)    

Add_names(result)

result.sort_values(by = ["Score"], ascending = False, inplace = True) 
result


# Main - part 2 : finding the best database

In [ ]:
best_scores_Uniprot = Best_scores_per_dtbase("UniProt Reference Proteomes", result)
best_scores_Ensembl = Best_scores_per_dtbase("Ensembl", result)
best_scores_RefSeq = Best_scores_per_dtbase("RefSeq", result)

l_dtbase = Col_in_list("source_db", result)

best_database_full = Best_dtbase_per_taxon(best_scores_Uniprot, best_scores_Ensembl, best_scores_RefSeq, l_dtbase, result)

best_database = best_database_full[0]
comp_database = best_database_full[1]

#best_database
comp_database 

In [ ]:
l_UP = [0] * len(comp_database)
l_Ens = [0] * len(comp_database)
l_RS = [0] * len(comp_database)
thres = 0.05   # Tuckey fence 

for index, row in comp_database.iterrows():
    if row["best_dtbase"] == "UniProt Reference Proteomes": 
        l_UP[index] = True
        if abs(row["diff_UP_Ens"]) < thres:
            l_Ens[index] = True
        else:
            l_Ens[index] = False
        if abs(row["diff_UP_RS"]) < thres: 
            l_RS[index] = True
        else:
            l_RS[index] = False
    elif row["best_dtbase"] == "Ensembl": 
        l_Ens[index] = True
        if abs(row["diff_UP_Ens"]) < thres:
            l_UP[index] = True
        else: 
            l_UP[index] = False
        if abs(row["diff_Ens_RS"]) < thres:
            l_RS[index] = True
        else:
            l_RS[index] = False
    elif row["best_dtbase"] == "RefSeq":
        l_RS[index] = True 
        if abs(row["diff_UP_RS"]) < thres:
            l_UP[index] = True
        else:
            l_UP[index] = False
        if abs(row["diff_Ens_RS"]) < thres: 
            l_Ens[index] = True
        else:
            l_Ens[index] = False 


arrays = [l_UP, l_Ens, l_RS]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=["UniProt", "Ensembl", "RefSeq"])
serie = pd.Series(index.value_counts(), index = index)

from upsetplot import UpSet
test = UpSet(serie, subset_size = "count", facecolor = "#3db7e9ff", show_counts = True).plot() #, show_percentages = True



In [ ]:
species_comp = pd.DataFrame()
species_comp["taxon_id"] = comp_database["taxon_id"]
Add_names(species_comp)
species_comp["UP"] = l_UP
species_comp["Ens"] = l_Ens
species_comp["RS"] = l_RS

Ens_solo = dict()
RefSeq_solo = dict()
Uniprot_solo = dict()
Ens_RefSeq = dict()
Uniprot_Ens = dict()
Uniprot_RefSeq = dict()

l_diff_spe = pd.DataFrame(columns = species_comp.columns)

for index, row in species_comp.iterrows():
    if row["UP"] == False or row["Ens"]== False or row["RS"] == False:
        l_diff_spe.loc[index] = row.tolist()

l_diff_spe

In [ ]:
cricetulus = pd.DataFrame(columns = result.columns)
mesocricetus = pd.DataFrame(columns = result.columns)
rattus = pd.DataFrame(columns = result.columns)
cricetulus_1 = pd.DataFrame(columns = result.columns)

for index, row in result.iterrows():
    if row["taxon_id"] == 10029:
        cricetulus.loc[index] = row.tolist()
    if row["taxon_id"] == 10036:
        mesocricetus.loc[index] = row.tolist()
    if row["taxon_id"] == 10116:
        rattus.loc[index] = row.tolist()

for index, row in result.iterrows():
    if row["taxon_id"] == 10029:
        cricetulus_1.loc[0] = row.tolist()

        
#10029 Cricetulus griseus 
#10036 Mesocricetus auratus 
#10116 Rattus norvegicus 
#pd.options.display.max_colwidth = 150
#cricetulus_1
cricetulus
#mesocricetus
#rattus

# liste des especes des catégories --> index des listes 

# Main - part 3 : get the highest diversity for the species choice

In [ ]:
def Species_choice(starting_species, best_database, k):


    tree = ncbi.get_topology(list(best_database["taxon_id"]), intermediate_nodes=True)
    tax2score = dict(zip(best_database.taxon_id, best_database.Score))
    leaves_left = [str(x) for x in list(best_database["taxon_id"])]
    subnodes = set()

    while len(starting_species) < k:
        for species in starting_species:
            if species in leaves_left:
                leaves_left.remove(species)
                node = tree&species
                for ancestor in node.get_ancestors():
                    subnodes.add(ancestor.name)
        #print(subnodes)
        #print(leaves_left)

        tree = ncbi.get_topology(list(leaves_left), intermediate_nodes=True)
        tree.convert_to_ultrametric()
        dist= list()
        mod_dist = float(0)
        b_div = float(0)

        for leaf in tree.iter_leaves():
            for ancest in leaf.iter_ancestors():
                if ancest.name in subnodes:
                    br_l = tree.get_distance(leaf, ancest.name, topology_only = False)
                    score_leaf = tax2score[int(leaf.name)] 
                    div = br_l * score_leaf
                    if div > b_div:
                        b_div = div
                        select_spec = leaf.name
                    break

        starting_species.append(str(select_spec))
    return(starting_species)


In [ ]:
#mettre tous les noeuds parents des espèces selectionnees dans un set
#pour les nouvelles especes ajoutter leurs parents 
#calcul des distances par rapport au plus proche noeud du set 
# garder en mémoire uniquement la meilleure distance dans la boucle qui recalcule toutes les dist 

In [ ]:
k = 5
starting_taxon = [str(x) for x in list(best_database["taxon_id"])]
starting_species = [starting_taxon[0]]
#starting_species=["10090"]
Species_choice(starting_species, best_database, k)

print(starting_species)

name_sel_species = ncbi.get_taxid_translator(starting_species)
name_sel_species
#mettre tous les noeuds parents des espèces selectionnees dans un set
#pour les nouvelles especes ajoutter leurs parents 
#calcul des distances par rapport au plus proche noeud du set 
# garder en mémoire uniquement la meilleure distance dans la boucle qui recalcule toutes les dist 


In [ ]:
# vérifier que la starting specie est une feuille 
# vérifier qu'il y a des données pour les 3 dtbases 

# Others

### CSV output

In [ ]:
#best_database.to_csv("/Users/perrinekergoat/Desktop/best_database.csv")
#comp_database.to_csv("/Users/perrinekergoat/Desktop/comp_database.csv")


### Graphical verification of scores

In [ ]:
data_plot = result["Score"]
data_plot = data_plot.values.tolist()
plt.plot(data_plot)
plt.title('Vertebrate scores')
plt.xlabel('Species')
plt.ylabel('Score')
plt.show()

In [ ]:
data_plot = best_database["Score"]
data_plot = data_plot.values.tolist()
plt.plot(data_plot)
plt.title('Vertebrate scores')
plt.xlabel('Species')
plt.ylabel('Score')
plt.show()

### Affichage graphique type OMark

In [ ]:
def OMark_graph (data):
    fig, axes = plt.subplots(2,1,figsize = (15,5))

    data.plot.bar(y=['hog_single', 'hog_duplicated','hog_missing'], label=['Single', 'Duplicated', 'Missing'], color = ['#46bea1ff', '#cfee8eff','#ed1c5aff'], 
                    ylim=(0,100), width=1.0,  stacked=True, ax=axes[0],edgecolor='gray', lw=0.1, rasterized=True)

    data.plot.bar(y=['prot_consistent_fit','prot_consistent_partial', 'prot_consistent_fragment', 
                               'prot_contamination_fit', 'prot_contamination_partial', 'prot_contamination_fragment',
                               'prot_inconsistent_fit', 'prot_inconsistent_partial', 'prot_inconsistent_fragment',
                               'prot_not_mapped'], label=['Consistent', '','','Contamination','','','Inconsistent','','','Unknown'],
                    color =['#3db7e9ff','#3db7e9ff','#3db7e9ff','#e69f00ff','#e69f00ff','#e69f00ff','#8a5df1ff','#8a5df1ff','#8a5df1ff','#000000ff' ],
                               edgecolor='white', lw=0.2, ylim=(0,100), width=1.0, stacked=True, ax=axes[1], rasterized=True)

    xticks = axes[0].get_xaxis().set_visible(False)
    xticks = axes[1].xaxis.get_major_ticks()
    
    axes[1].set_xticklabels(data['sc_name'].squeeze())#data['source_db'].squeeze()) "Cricetulus"

    bars = axes[1].patches
    hatches= ['','///','///','','///','///','','///','///','']
    col = ['white', 'darkslategray', 'whitesmoke','white', 'darkslategray', 'whitesmoke','white', 'darkslategray', 'whitesmoke','white']
    for index, bar  in enumerate(bars):
        bar.set_hatch(hatches[index//len(data)])
        bar.set_edgecolor(col[index//len(data)])

    partial_patch = mpatches.Patch(facecolor='white',lw='0.1', hatch='///', edgecolor='darkslategray', label='Partial mapping', alpha=.99)
    fragment_patch = mpatches.Patch(facecolor='darkslategray',lw='0.1', hatch='///', edgecolor='white', label='Fragments', alpha=.99)
    custom_legend = [partial_patch,fragment_patch]
    axes[0].legend(title='Completeness', loc='center left', bbox_to_anchor=(1, 0.5))
    axes[1].legend(title='Consistency', handles=axes[1].get_legend_handles_labels()[0]+custom_legend, loc='center left', bbox_to_anchor=(1, 0.5))


    axes[1].set_xlabel('Species') #Cricetulus griseus
    axes[0].set_ylabel('Proportion of conserved HOGs')
    axes[1].set_ylabel('Proportion of proteomes')
    axes[1].set_ylim(axes[1].get_ylim()[::-1])
    axes[1].tick_params(axis='x',labelsize=10)

    plt.subplots_adjust(wspace=0, hspace=0.05)

    plt.show()
    



In [ ]:
#
result.sort_values(by = "taxon_id", inplace = True, ascending = True)
best_database.sort_values(by = "taxon_id", inplace = True, ascending = True)

#result.sort_index()

#OMark_graph(result)

#
OMark_graph(best_database)

#OMark_graph(cricetulus_1)
#OMark_graph(mesocricetus)
#OMark_graph(rattus)


#10029 Cricetulus griseus 
#10036 Mesocricetus auratus 
#10116 Rattus norvegicus 

### Set rows' index as taxonomic id 

In [ ]:
#Data = Data.set_index("taxon_id", drop = False)
#Data

#pas une bonne idée :  perturbe tout à cause des index 

# Brouilllons 

Ensuite algo optimiser le chemin dans l'arbre --> NOA'S ARC PROBLEM 

ETE3 package: .gettopology et .convert_to_ultrametric

faire avec greedy algorithm

In [ ]:
from platform import python_version
 
 
print("Current Python Version-", python_version())